In [13]:
%pip install pyautogen


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
%pip install groq
%pip install dask
from groq import Groq


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
import json
import os
from typing import Literal

from typing_extensions import Annotated

import autogen

config_list = [
    {"api_type": "groq", "model": "llama3-groq-70b-8192-tool-use-preview", "api_key": "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", "cache_seed": None}
]

In [16]:
# Function to convert MongoDB extended JSON types to standard Python types
def convert_mongo_json(data):
    if isinstance(data, dict):
        if "$numberInt" in data:
            return int(data["$numberInt"])
        return {key: convert_mongo_json(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_mongo_json(item) for item in data]
    return data

# Create the chatbot agent
chatbot = autogen.AssistantAgent(
    name="chatbot",
    system_message="""For tasks related to risk tolerance, follow the functions provided to analyze user data.""",
    llm_config={"config_list": config_list},
)

# Create a user proxy agent
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and "HAVE FUN!" in x.get("content", ""),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
    code_execution_config=False,
)

# Load JSON data
with open("userForm.json", "r") as file:
    raw_user_data = json.load(file)

# Convert MongoDB extended JSON to standard Python types
user_data = convert_mongo_json(raw_user_data)

# Demographic Agent
@user_proxy.register_for_execution()
@chatbot.register_for_llm(description="Analyze user's demographic information.")
def demographic_analysis(
    age: Annotated[int, "User's age"], 
    occupation: Annotated[str, "Occupation"]
) -> str:
    return f"User's age is {age}, and occupation is {occupation}."

# Financial Agent
@user_proxy.register_for_execution()
@chatbot.register_for_llm(description="Analyze user's financial information.")
def financial_analysis(
    income: Annotated[int, "Annual income"], 
    debt: Annotated[int, "Debt"]
) -> str:
    return f"User's income is {income}, with debt of {debt}."

# Risk Tolerance Agent
@user_proxy.register_for_execution()
@chatbot.register_for_llm(description="Reassess user's risk tolerance.")
def risk_tolerance_analysis(
    investment: Annotated[int, "Initial investment"], 
    target: Annotated[int, "Target value"]
) -> str:
    tolerance = (target - investment) / investment * 100
    return f"Updated risk tolerance is {tolerance}%."

# Compile results
@user_proxy.register_for_execution()
@chatbot.register_for_llm(description="Compile and write updated data.")
def compile_and_write(updated_data: Annotated[dict, "Updated JSON data"]) -> str:
    with open("updated_userForm(llama70B).json", "w") as updated_file:
        json.dump(updated_data, updated_file, indent=4)
    return "New JSON file with updated risk tolerance created."

# Extract relevant data from JSON
try:
    age = int(user_data['userDetails']['demographicInformation']['data']['age'])
    occupation = user_data['userDetails']['demographicInformation']['data']['occupation']
    income = int(user_data['userDetails']['financialInformation']['data']['income'])
    debt = int(user_data['userDetails']['financialInformation']['data']['debt'])
    investment = int(user_data['riskTolerance']['parameters']['data']['initialInvestment'])
    target = int(user_data['riskTolerance']['parameters']['data']['targetValue'])
    
    # Ensure data is in correct format
    if not all(isinstance(x, int) for x in [age, income, debt, investment, target]):
        raise ValueError("One or more values are not integers.")
except KeyError as e:
    print(f"Error extracting data: Missing key {e}")
    age = occupation = income = debt = investment = target = None
except ValueError as e:
    print(f"Error converting data: {e}")
    age = occupation = income = debt = investment = target = None

if None not in [age, occupation, income, debt, investment, target]:
    # Start analysis
    res = user_proxy.initiate_chat(
        chatbot,
        message=f"Analyze the user's demographic (age: {age}, occupation: {occupation}), financials (income: {income}, debt: {debt}), and risk tolerance (investment: {investment}, target: {target}), then update the risk tolerance value.",
        summary_method="reflection_with_llm",
    )

    print(f"LLM SUMMARY: {res.summary['content']}")
else:
    print("Data extraction failed. Check the JSON file structure and values.")

user_proxy (to chatbot):

Analyze the user's demographic (age: 25, occupation: Software Engineer), financials (income: 1000000, debt: 100000), and risk tolerance (investment: 1000000, target: 2000000), then update the risk tolerance value.

--------------------------------------------------------------------------------
chatbot (to user_proxy):

***** Suggested tool call (call_n43w): demographic_analysis *****
Arguments: 
{"age": 25, "occupation": "Software Engineer"}
*****************************************************************
***** Suggested tool call (call_p21q): financial_analysis *****
Arguments: 
{"income": 1000000, "debt": 100000}
***************************************************************
***** Suggested tool call (call_003v): risk_tolerance_analysis *****
Arguments: 
{"investment": 1000000, "target": 2000000}
********************************************************************
***** Suggested tool call (call_zpbe): compile_and_write *****
Arguments: 
{"updated_data": 

## LLM - llama-3.1-70b-versatile

In [17]:
config_list = [
    {"api_type": "groq", "model": "llama-3.1-70b-versatile", "api_key": "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", "cache_seed": None}
]

In [18]:
# Function to convert MongoDB extended JSON types to standard Python types
def convert_mongo_json(data):
    if isinstance(data, dict):
        if "$numberInt" in data:
            return int(data["$numberInt"])
        return {key: convert_mongo_json(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_mongo_json(item) for item in data]
    return data

# Create the chatbot agent
chatbot = autogen.AssistantAgent(
    name="chatbot",
    system_message="""For tasks related to risk tolerance, follow the functions provided to analyze user data.""",
    llm_config={"config_list": config_list},
)

# Create a user proxy agent
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and "HAVE FUN!" in x.get("content", ""),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
    code_execution_config=False,
)

# Load JSON data
with open("userForm.json", "r") as file:
    raw_user_data = json.load(file)

# Convert MongoDB extended JSON to standard Python types
user_data = convert_mongo_json(raw_user_data)

# Demographic Agent
@user_proxy.register_for_execution()
@chatbot.register_for_llm(description="Analyze user's demographic information.")
def demographic_analysis(
    age: Annotated[int, "User's age"], 
    occupation: Annotated[str, "Occupation"]
) -> str:
    return f"User's age is {age}, and occupation is {occupation}."

# Financial Agent
@user_proxy.register_for_execution()
@chatbot.register_for_llm(description="Analyze user's financial information.")
def financial_analysis(
    income: Annotated[int, "Annual income"], 
    debt: Annotated[int, "Debt"]
) -> str:
    return f"User's income is {income}, with debt of {debt}."

# Risk Tolerance Agent
@user_proxy.register_for_execution()
@chatbot.register_for_llm(description="Reassess user's risk tolerance.")
def risk_tolerance_analysis(
    investment: Annotated[int, "Initial investment"], 
    target: Annotated[int, "Target value"]
) -> str:
    tolerance = (target - investment) / investment * 100
    return f"Updated risk tolerance is {tolerance}%."

# Compile results
@user_proxy.register_for_execution()
@chatbot.register_for_llm(description="Compile and write updated data.")
def compile_and_write(updated_data: Annotated[dict, "Updated JSON data"]) -> str:
    with open("updated_userForm(llama 3.1 70B).json", "w") as updated_file:
        json.dump(updated_data, updated_file, indent=4)
    return "New JSON file with updated risk tolerance created."

# Extract relevant data from JSON
try:
    age = int(user_data['userDetails']['demographicInformation']['data']['age'])
    occupation = user_data['userDetails']['demographicInformation']['data']['occupation']
    income = int(user_data['userDetails']['financialInformation']['data']['income'])
    debt = int(user_data['userDetails']['financialInformation']['data']['debt'])
    investment = int(user_data['riskTolerance']['parameters']['data']['initialInvestment'])
    target = int(user_data['riskTolerance']['parameters']['data']['targetValue'])
    
    # Ensure data is in correct format
    if not all(isinstance(x, int) for x in [age, income, debt, investment, target]):
        raise ValueError("One or more values are not integers.")
except KeyError as e:
    print(f"Error extracting data: Missing key {e}")
    age = occupation = income = debt = investment = target = None
except ValueError as e:
    print(f"Error converting data: {e}")
    age = occupation = income = debt = investment = target = None

if None not in [age, occupation, income, debt, investment, target]:
    # Start analysis
    res = user_proxy.initiate_chat(
        chatbot,
        message=f"Analyze the user's demographic (age: {age}, occupation: {occupation}), financials (income: {income}, debt: {debt}), and risk tolerance (investment: {investment}, target: {target}), then update the risk tolerance value.",
        summary_method="reflection_with_llm",
    )

    print(f"LLM SUMMARY: {res.summary['content']}")
else:
    print("Data extraction failed. Check the JSON file structure and values.")

user_proxy (to chatbot):

Analyze the user's demographic (age: 25, occupation: Software Engineer), financials (income: 1000000, debt: 100000), and risk tolerance (investment: 1000000, target: 2000000), then update the risk tolerance value.

--------------------------------------------------------------------------------
chatbot (to user_proxy):

***** Suggested tool call (call_yh3h): demographic_analysis *****
Arguments: 
{"age": 25, "occupation": "Software Engineer"}
*****************************************************************
***** Suggested tool call (call_qkmw): financial_analysis *****
Arguments: 
{"income": 100000, "debt": 100000}
***************************************************************
***** Suggested tool call (call_d600): risk_tolerance_analysis *****
Arguments: 
{"investment": 1000000, "target": 2000000}
********************************************************************
***** Suggested tool call (call_yaqj): risk_tolerance_analysis *****
Arguments: 
{"investment